# <font color='red' style='font-size: 30px;'> Rede Multinível </font>
<hr style='border: 2px solid red;'>

## <font color = 'black' style='font-size: 26px;'> Imports </font>  
<hr style = 'border: 1.5px solid black;'>

In [1]:
# Libs Utilizadas
import networkx as nx
from sklearn.metrics import mean_squared_error
import random
import numpy as np

In [2]:
random.seed(42)

## <font color = 'black' style='font-size: 26px;'> Funções Grafos </font>  
<hr style = 'border: 1.5px solid black;'>

In [3]:
def inicializa_grafo(populacao, porcentagem_adultos):
    
    # Criando o Grafo
    grafo = nx.Graph()
    
    # Quantidade de nós a serem instanciados
    total_adultos = round(porcentagem_adultos * populacao)
 
    for i in range(populacao):
        if i < total_adultos:
            grafo.add_node(i, classe="Adulto")
        else:
            grafo.add_node(i, classe="Criança")
    
    return grafo

In [4]:
def cria_arestas(grafo, lista_nos, prob_criacao_arestas, peso):
    
    # Cria arestas entre todos os nós da lista
    for i in range(len(lista_nos) - 1):
        for j in range(i + 1, len(lista_nos)):
            if random.random() < prob_criacao_arestas:
                grafo.add_edge(lista_nos[i], lista_nos[j], peso=peso)
                
    return grafo

## <font color = 'black' style='font-size: 26px;'> Camada "Família" </font>  
<hr style = 'border: 1.5px solid black;'>

In [5]:
def contabiliza_nos(grafo):
    adultos_nao_agrupados= []
    criancas_nao_agrupadas = []    
    for i, j in grafo.nodes.items():
        if j['classe'] == "Adulto":
            adultos_nao_agrupados.append(i)
        else:
            criancas_nao_agrupadas.append(i)
            
    return adultos_nao_agrupados, criancas_nao_agrupadas

In [6]:
def separa_adultos(porc_tam_familia, populacao, adultos_nao_agrupados):    
    total = 0
    for i, j in porc_tam_familia.items():
        total += round(j * populacao / int(i))    
    adultos_separados = adultos_nao_agrupados[0:total]
    del adultos_nao_agrupados[0:total]
    
    return adultos_separados

In [7]:
def gera_camada_familia(grafo, porc_tam_familia, peso):
    
    # Contabilização dos nós de cada classe
    adultos_nao_agrupados, criancas_nao_agrupadas = contabiliza_nos(grafo)
    
    # Separa 1 adulto para compor cada família 
    populacao = len(adultos_nao_agrupados) + len(criancas_nao_agrupadas)
    adultos_separados = separa_adultos(porc_tam_familia, populacao, adultos_nao_agrupados)    

    # Familias com 1 integrante    
    solo = round(porc_tam_familia['1'] * populacao)
    del adultos_separados[0:solo]
    del porc_tam_familia['1']

    # Agrupando demais familias
    adultos_e_criancas = adultos_nao_agrupados + criancas_nao_agrupadas
    random.shuffle(adultos_e_criancas)
    tamanho = 2
        
    for i in porc_tam_familia.values():
        
        for _ in range(round((i * populacao) / tamanho)):            
            aux = list()
            aux.append(adultos_separados.pop())
            
            for _ in range(tamanho - 1):
                # Como os valores são arredondados, pode faltar elemento, causando erro de remoção de lista vazia 
                if len(adultos_e_criancas):                
                    aux.append(adultos_e_criancas.pop())
            grafo = cria_arestas(grafo, aux, 1, peso)
        tamanho += 1
    
    # Agrupa os nós que sobraram em um único conjunto
    if adultos_separados or adultos_e_criancas:
        grafo = cria_arestas(grafo, adultos_separados + adultos_e_criancas, 1, peso)
        
    return grafo

## <font color = 'black' style='font-size: 26px;'> Demais Camadas </font>  
<hr style = 'border: 1.5px solid black;'>

In [8]:
def remove_elem(lista, list_elem_removidos):
    for i in list_elem_removidos:
        lista.remove(i)

In [9]:
def separa_nos(grafo, crit_agrup, valor_agrup):
    
    if valor_agrup == 'Todos':
        nos = []
        for i, j in grafo.nodes.items():
            nos.append(i)
        return nos
    else:
        nos = []
        for i, j in grafo.nodes.items():
            if j[crit_agrup] == valor_agrup:
                nos.append(i)
        return nos        

In [10]:
def gera_camada(grafo, crit_agrup, valor_agrup, porc_agrup, lim_inf, lim_sup, prob_criacao_arestas, peso):
    
    # Seleciona os nós
    nos = separa_nos(grafo, crit_agrup, valor_agrup)
    
    # Separa nós a serem utilizados
    total_nos_agrup = round(len(nos) * porc_agrup)
    nos_agrupamento = random.sample(nos, total_nos_agrup)

    while(len(nos_agrupamento)):
        
        # Tamanho randdômico dos agrupamentos
        tam_grupo= random.randint(lim_inf, lim_sup)
        
        if len(nos_agrupamento) > tam_grupo:            
            grupo =  random.sample(nos_agrupamento, tam_grupo)
            cria_arestas(grafo, grupo, prob_criacao_arestas, peso)
            remove_elem(nos_agrupamento, grupo)
            
        else:
            cria_arestas(grafo, nos_agrupamento, prob_criacao_arestas, peso)
            nos_agrupamento.clear()
            
    return grafo

## <font color = 'black' style='font-size: 26px;'> Gerando Rede </font>  
<hr style = 'border: 1.5px solid black;'>

In [11]:
# Inicializando o Grafo
grafo = inicializa_grafo(1000, 0.76)

In [12]:
porc_tam_familia = {'1' : 0.12, 
                    '2' : 0.22, 
                    '3' : 0.25, 
                    '4' : 0.21, 
                    '5' : 0.11, 
                    '6' : 0.05, 
                    '7' : 0.02,
                    '8' : 0.01,
                    '9' : 0.005,
                    '10' : 0.005}

In [13]:
# Camada Família
grafo = gera_camada_familia(grafo, porc_tam_familia, 0.125)

# Camada Trabalho
grafo = gera_camada(grafo, 'classe', 'Adulto', 1, 5, 30, 0.1, 0.238)

# Camada Escola
grafo = gera_camada(grafo, 'classe', 'Criança', 0.6, 16, 30, 0.1, 0.12)

# Camada Transporte
grafo = gera_camada(grafo, 'classe', 'Todos', 0.5, 10, 30, 0.1, 0.05)

# Camada Religião
grafo = gera_camada(grafo, 'classe', 'Todos', 0.4, 10, 100, 0.1, 0.01)

# Camada Aleatória
grafo = gera_camada(grafo, 'classe', 'Todos', 1, 3, 3, 1, 0.006)
nx.write_graphml(grafo, "grafo.graphml")

In [14]:
cont = []
for _, _, i in grafo.edges.data():
    cont.append(i['peso'])

In [15]:
from collections import Counter 
c = Counter(cont)
c

Counter({0.238: 761,
         0.01: 1114,
         0.006: 999,
         0.05: 544,
         0.125: 1152,
         0.12: 125})

## <font color = 'black' style='font-size: 26px;'> Simulador </font>  
<hr style = 'border: 1.5px solid black;'>

In [16]:
def simulacao(grafo, dias, beta, gamma, nos_infectados, nos_recuperados):
    
    # Composite Model instantiation
    model = gc.CompositeModel(grafo)

    # Model statuses
    model.add_status("Susceptible")
    model.add_status("Infected")
    model.add_status("Removed")
    
    # Compartment definition
    c1 = es.EdgeStochastic(triggering_status="Infected")
    c2 = es.NodeStochastic(gamma)
    
    # Rule definition    
    model.add_rule("Susceptible", "Infected", c1)
    model.add_rule("Infected", "Removed", c2)
    
    # Model initial status configuration
    config = mc.Configuration()
    
    # Threshold specs
    for e, data in grafo.edges.items():
        threshold = beta * data['peso']
        config.add_edge_configuration("threshold", e, threshold)

    # Inicializando os nos infectados
    config.add_model_initial_configuration("Infected", nos_infectados)
    config.add_model_initial_configuration("Removed", nos_recuperados)

    # Simulation execution
    model.set_initial_status(config)
    iterations = model.iteration_bunch(100)    
    return iterations

In [17]:
for num, i in enumerate(nx.degree_histogram(grafo)):
    print(f'{num}: {i}')

0: 0
1: 0
2: 7
3: 37
4: 55
5: 81
6: 92
7: 98
8: 99
9: 109
10: 68
11: 64
12: 69
13: 60
14: 39
15: 38
16: 25
17: 15
18: 15
19: 12
20: 2
21: 7
22: 4
23: 3
24: 1
